In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import re

def scrape_links():
    user_input_url = url_entry.get()
    
    try:
        response = requests.get(user_input_url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            links = soup.find_all('a', href=True)

            result_text.delete(1.0, tk.END)
            
            # Counter for line numbers
            line_number = 1

            for link in links:
                href = link['href']
                absolute_url = urljoin(user_input_url, href)

                if urlparse(absolute_url).netloc == urlparse(user_input_url).netloc:
                    # Append line number and URL to the result text
                    result_text.insert(tk.END, f"{line_number}. {absolute_url}\n")
                    line_number += 1
        else:
            result_text.delete(1.0, tk.END)
            result_text.insert(tk.END, f"Failed to retrieve the page. Status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        result_text.delete(1.0, tk.END)
        result_text.insert(tk.END, f"An error occurred while making the request: {e}")

def scrape_contact_info():
    user_input_url = url_entry.get()

    try:
        response = requests.get(user_input_url, verify=False)

        if response.status_code == 200:
            email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b')
            phone_pattern = re.compile(r'\b(?:\+\d{1,3}\s?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b')
            fax_pattern = re.compile(r'Fax:\s?\b(?:\+\d{1,3}\s?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b')

            contact_info = set(re.findall(email_pattern, response.text))
            contact_info.update(('number phone=' + number for number in re.findall(phone_pattern, response.text)))
            contact_info.update(('number fax=' + number for number in re.findall(fax_pattern, response.text)))

            result_text.insert(tk.END, "\n\nContact Information:\n")

            for info in contact_info:
                result_text.insert(tk.END, f"{info}\n")

        else:
            result_text.delete(1.0, tk.END)
            result_text.insert(tk.END, f"Failed to retrieve the page. Status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        result_text.delete(1.0, tk.END)
        result_text.insert(tk.END, f"An error occurred while making the request: {e}")

def count_occurrences():
    user_input_url = url_entry.get()
    user_input_word = word_entry.get()

    try:
        response = requests.get(user_input_url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            content_text = soup.get_text()

            # Count occurrences of the specified word
            word_count = content_text.lower().count(user_input_word.lower())

            result_text.delete(1.0, tk.END)
            result_text.insert(tk.END, f"Occurrences of '{user_input_word}' on the page: {word_count}\n")
        else:
            result_text.delete(1.0, tk.END)
            result_text.insert(tk.END, f"Failed to retrieve the page. Status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        result_text.delete(1.0, tk.END)
        result_text.insert(tk.END, f"An error occurred while making the request: {e}")

def save_results():
    results = result_text.get(1.0, tk.END)
    
    file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text Files", "*.txt")])
    
    if file_path:
        with open(file_path, 'w') as file:
            file.write(results)

def clear_results():
    # Clear the contents of the result text widget
    result_text.delete(1.0, tk.END)

root = tk.Tk()
root.title("Web Scraping Tool")

# Create a label for URL input
url_label = ttk.Label(root, text="Enter URL:", font=("Arial", 12))
url_label.pack(pady=10)  # Add padding

# Create a custom style for the URL entry
url_entry_style = ttk.Style()
url_entry_style.configure("URL.TEntry", font=("Arial", 12), width=50)  # Set font and width

# Create a URL entry with the custom style
url_entry = ttk.Entry(root, style="URL.TEntry")
url_entry.pack()

# Create a custom style for the buttons
button_style = ttk.Style()
button_style.configure("Custom.TButton", font=("Arial", 12), background="green", width=15)  # Set font, background, and width

# Create a button to scrape links
scrape_button = ttk.Button(root, text="Scrape Links", command=scrape_links, style="Custom.TButton")
scrape_button.pack(pady=10)  # Add padding

# Create a button to scrape contact information
scrape_contact_button = ttk.Button(root, text="Scrape Contact Info", command=scrape_contact_info, style="Custom.TButton")
scrape_contact_button.pack(pady=10)  # Add padding

# Create a label for word input
word_label = ttk.Label(root, text="Enter Word:", font=("Arial", 12))
word_label.pack(pady=10)  # Add padding

# Create a word entry with the custom style
word_entry = ttk.Entry(root, style="URL.TEntry")
word_entry.pack()

# Create a button to count occurrences
count_occurrences_button = ttk.Button(root, text="Count Occurrences", command=count_occurrences, style="Custom.TButton")
count_occurrences_button.pack(pady=10)  # Add padding

# Create a button to save results
save_button = ttk.Button(root, text="Save Results", command=save_results, style="Custom.TButton")
save_button.pack()

# Create a button to clear results
clear_button = ttk.Button(root, text="Clear Results", command=clear_results, style="Custom.TButton")
clear_button.pack()

# Create a text widget with a custom background color
result_text = tk.Text(root, height=15, width=70, bg="lightyellow")
result_text.pack()

root.mainloop()
